In [20]:
import numpy as np
import pandas as pd
import pickle

from sklearn.model_selection import StratifiedKFold

import lightgbm as lgb
from sklearn.metrics import roc_auc_score

In [2]:
train = pickle.load(open("../data/train_vectorized.pkl","rb"))
test = pickle.load(open("../data/test_vectorized.pkl","rb"))

In [12]:
train = pd.DataFrame(train)
test = pd.DataFrame(test)

In [13]:
train['IsTrain'] = np.int8(1)
test['IsTrain'] = np.int8(0)
train_test = pd.concat([train,test],axis=0)

In [14]:
del train,test

In [17]:
y = train_test['IsTrain']; train_test.drop(columns=['IsTrain'], inplace=True)

In [18]:
param = {'num_leaves': 200,
         'min_data_in_leaf': 60, 
         'objective':'binary',
         'max_depth': -1,
         'learning_rate': 0.1,
         "boosting": "gbdt",
         "feature_fraction": 0.8,
         "bagging_freq": 1,
         "bagging_fraction": 0.8,
         "bagging_seed": 11,
         "metric": 'auc',
         "lambda_l1": 0.1,
         "random_state": 7,
         "verbosity": -1,
         "n_jobs":-1}

clf = lgb.LGBMClassifier(**param)

In [21]:
skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 7)
cols = [c for c in train_test.columns]

In [22]:
feature_importance_df = pd.DataFrame()
unsure_train_data = pd.DataFrame()
for f_, (train_index, test_index) in enumerate(skf.split(train_test, y)):
        x_train, x_test = train_test.iloc[train_index][cols], train_test.iloc[test_index][cols]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        clf.fit(x_train.values, y_train.values)#, eval_set=[(x1, y1)],
               #eval_metric='logloss', verbose=False,early_stopping_rounds=10)
                
        prval = clf.predict_proba(x_test)[:,1]
        
        unsure_train_data = x_test[(y_test == 1) & (prval < .7)]
        
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = cols
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = f_ + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        
        print(roc_auc_score(y_test,prval))

0.5313719951937479
0.5143579992492291
0.5131338366111635
0.5141118905239574
0.5252724858271551


In [23]:
unsure_train_data.shape

(3399, 200)

In [31]:
arr = np.array(unsure_train_data.index.values)
pickle.dump(arr, open('../Data/validation_index.pkl','wb'))

In [29]:
arr

array([    7,    10,    14, ..., 18795, 18824, 18827], dtype=int64)